# V2log YOLO11 Weight Plate Training

**셀을 위에서 아래로 순서대로 ▶ 클릭하면 됩니다!**

| 셀 | 내용 | 예상 시간 |
|-----|------|----------|
| 1 | Google Drive 연결 | 30초 |
| 2 | 데이터셋 압축 해제 | 1분 |
| 3 | YOLO 설치 + 설정 | 1분 |
| 4 | **학습 시작** | **30분~2시간** |
| 5 | 모델 변환 + 저장 | 2분 |

⚠️ **주의: 학습 중 브라우저 탭 닫지 마세요!**

## 셀 1: Google Drive 연결
▶ 실행 → 팝업에서 Google 계정 선택 → 허용

In [1]:
from google.colab import drive
drive.mount('/content/drive')
print('Drive 연결 완료!')

Mounted at /content/drive
Drive 연결 완료!


## 셀 2: 데이터셋 압축 해제
Drive에 업로드한 ZIP을 Colab 로컬에 풀기

In [2]:
import zipfile
import os

# ZIP 파일 경로
zip_path = '/content/drive/MyDrive/V2log-CV-Training/v2log-weight-plates.v2i.yolov8.zip'

# 파일 존재 확인
if not os.path.exists(zip_path):
    print('\n ZIP 파일을 못 찾았습니다!')
    print(f' 확인할 경로: {zip_path}')
    print('\n Google Drive에서 실제 경로를 확인하세요:')
    drive_path = '/content/drive/MyDrive/V2log-CV-Training/'
    if os.path.exists(drive_path):
        print(f' {drive_path} 안의 파일들:')
        for f in os.listdir(drive_path):
            print(f'   {f}')
    else:
        print(f' {drive_path} 폴더가 없습니다!')
        print(' Drive 루트 폴더 목록:')
        for f in os.listdir('/content/drive/MyDrive/')[:20]:
            print(f'   {f}')
else:
    # 압축 해제
    print(f'ZIP 발견: {zip_path}')
    print('압축 해제 중...')
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')

    # 폴더 구조 확인
    print('\n폴더 구조:')
    for root, dirs, files in os.walk('/content/dataset'):
        level = root.replace('/content/dataset', '').count(os.sep)
        indent = '  ' * level
        print(f'{indent}{os.path.basename(root)}/')
        if level < 2:
            for file in files[:3]:
                print(f'{indent}  {file}')
            if len(files) > 3:
                print(f'{indent}  ... 외 {len(files)-3}개')

    # 이미지 개수 확인
    for split in ['train', 'valid', 'test']:
        img_dir = f'/content/dataset/{split}/images'
        if os.path.exists(img_dir):
            count = len(os.listdir(img_dir))
            print(f'\n{split}: {count}장')

    print('\n데이터셋 준비 완료!')

ZIP 발견: /content/drive/MyDrive/V2log-CV-Training/v2log-weight-plates.v2i.yolov8.zip
압축 해제 중...

폴더 구조:
dataset/
  data.yaml
  README.roboflow.txt
  train/
    labels/
    images/
  valid/
    labels/
    images/
  test/
    labels/
    images/

train: 648장

valid: 187장

test: 91장

데이터셋 준비 완료!


## 셀 3: YOLO 설치 + data.yaml 경로 수정

In [3]:
# Ultralytics (YOLO) 설치
!pip install ultralytics -q

import yaml

# data.yaml 경로 수정 (Colab 환경에 맞게)
yaml_path = '/content/dataset/data.yaml'

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# 경로를 Colab 절대 경로로 수정
data['path'] = '/content/dataset'
data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print('YOLO 설치 완료!')
print(f'\n클래스 수: {data["nc"]}')
print(f'클래스: {data["names"]}')
print('\ndata.yaml 경로 수정 완료!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.9 MB/s eta 0:00:00
YOLO 설치 완료!

클래스 수: 5
클래스: ['plate_10kg', 'plate_15kg', 'plate_2.5kg', 'plate_20kg', 'plate_5kg']

data.yaml 경로 수정 완료!


## 셀 4: YOLO 학습 시작

⏱️ **30분~2시간 걸립니다. 브라우저 탭 닫지 마세요!**

화면에 epoch 1/100, 2/100 ... 이렇게 진행 상황이 나옵니다.

In [4]:
from ultralytics import YOLO

# YOLO 모델 로드 (자동 다운로드됨)
# yolo26n이 아직 없으면 yolo11n 사용
try:
    model = YOLO('yolo11n.pt')
    print('YOLO11n 모델 로드 완료')
except:
    model = YOLO('yolov8n.pt')
    print('YOLOv8n 모델 로드 완료 (fallback)')

# 학습 시작!
results = model.train(
    data='/content/dataset/data.yaml',
    epochs=100,        # 최대 100번 반복
    imgsz=640,         # 이미지 크기
    batch=16,          # 한 번에 16장씩
    device=0,          # GPU 사용
    patience=20,       # 20번 동안 개선 없으면 조기 종료
    save=True,         # 체크포인트 저장
    project='/content/models',
    name='weight_plate',
    exist_ok=True,     # 같은 이름 있어도 OK
)

# 결과 출력
print(f'\n{"="*50}')
print(f'학습 완료!')
print(f'mAP50: {results.results_dict.get("metrics/mAP50(B)", "N/A")}')
print(f'mAP50-95: {results.results_dict.get("metrics/mAP50-95(B)", "N/A")}')
print(f'{"="*50}')

mAP50 = results.results_dict.get('metrics/mAP50(B)', 0)
if mAP50 >= 0.8:
    print('\n 80% 이상! 다음 셀(변환)을 실행하세요!')
elif mAP50 >= 0.6:
    print('\n 60~80% — 나쁘지 않음. 일단 변환해서 테스트해보세요.')
else:
    print('\n 60% 미만 — 데이터 추가가 필요할 수 있습니다.')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLO11n 모델 로드 완료
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, img

## 셀 5: TFLite 변환 + Google Drive에 저장

학습 완료 후 실행! 결과물이 Google Drive에 자동 저장됩니다.

In [5]:
import shutil
import os
from ultralytics import YOLO

# 학습된 모델 로드
best_model_path = '/content/models/weight_plate/weights/best.pt'
model = YOLO(best_model_path)

# TFLite로 변환 (모바일용)
print('TFLite 변환 중...')
model.export(format='tflite', half=True, imgsz=640)
print('변환 완료!')

# Google Drive에 결과물 복사
dst = '/content/drive/MyDrive/V2log-CV-Training/results/'
os.makedirs(dst, exist_ok=True)

# 모델 파일 복사
weights_dir = '/content/models/weight_plate/weights/'
for f in os.listdir(weights_dir):
    src_file = os.path.join(weights_dir, f)
    if os.path.isfile(src_file):
        shutil.copy2(src_file, dst)
        size_mb = os.path.getsize(src_file) / (1024*1024)
        print(f'  저장: {f} ({size_mb:.1f} MB)')

# 학습 결과 그래프/CSV 복사
results_dir = '/content/models/weight_plate/'
for f in os.listdir(results_dir):
    src_file = os.path.join(results_dir, f)
    if os.path.isfile(src_file) and (f.endswith('.png') or f.endswith('.csv')):
        shutil.copy2(src_file, dst)
        print(f'  저장: {f}')

print(f'\n모든 결과물이 Google Drive에 저장되었습니다!')
print(f'위치: Google Drive > V2log-CV-Training > results')
print(f'\n최종 결과물: best_float16.tflite')
print(f'이 파일을 V2log 앱에 넣으면 무게 자동 감지 완성!')

TFLite 변환 중...
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n summary (fused): 101 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/content/models/weight_plate/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (5.2 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx>=1.12.0,<2.0.0', 'onnx2tf>=1.26.3,<1.29.0', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 7.75s
Prepared 9 packages in 3.86s
Installed 9 packages in 252ms
 + ai-edge-litert==2.1.2
 + backports-strenum==1.3.1
 + colorama==0.4.6
 + onnx==1.20.1
 + onnx-graphsurgeon==0.5.8
 + onnx2tf==1.28.8
 + onnxruntime-gpu=

/usr/local/lib/python3.12/dist-packages/torch/onnx/_internal/torchscript_exporter/utils.py:1447: OnnxExporterWarning: Exporting to ONNX opset version 22 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 
  warnings.warn(


ONNX: slimming with onnxslim 0.1.85...
ONNX: export success ✅ 1.8s, saved as '/content/models/weight_plate/weights/best.onnx' (10.2 MB)
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100% ━━━━━━━━━━━━ 1/1 16.1files/s 0.1s
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.8...
Saved artifact at '/content/models/weight_plate/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 9, 8400), dtype=tf.float32, name=None)
Captures:
  136760616481232: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  136760616479696: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  136760616480464: TensorSpec(shape=(16,), dtype=tf.float32, name=None)
  136760596415696: TensorSpec(shape=(4, 2), dtype=tf.int32, name=N

---
## 완료!

Google Drive > `V2log-CV-Training` > `results` 폴더에 결과물이 저장되었습니다.

| 파일 | 용도 |
|------|------|
| `best.pt` | 학습된 모델 (PC용) |
| `best_float16.tflite` | **앱에 넣을 최종 모델** |
| `results.png` | 학습 그래프 |
| `confusion_matrix.png` | 클래스별 정확도 |

**다음 단계**: `best_float16.tflite`를 V2log 앱 `assets/models/`에 넣고 Phase 2B 시작!